Importing needed libraries and changing the directory to the corpus directory to process them

In [9]:
from nltk.stem import PorterStemmer
from collections import Counter
from collections import OrderedDict
import os
import re
path = os.getcwd()
os.chdir("..")
#change to the corpus directory
path = 'corpus'
try:
    os.chdir(path)
except FileNotFoundError:
    print("Directory: {0} does not exist".format(path))
except NotADirectoryError:
    print("{0} is not a directory".format(path))
except PermissionError:
    print("You do not have permissions to change to {0}".format(path))


Creating mappings from files names to docIDs and vice-versa.  
Removing words such as: ACT 1, SCENE 1, redundant beginning metadata, single chars, punctuations.  
Splitting the whole data into tokens, and storing them as a dictionary with key as title and value as a list of tokens.  
Scanning the whole corpus, counting their frequencies, and removing top 30 most common words. So this is corpus specific stopword removal.  

In [10]:
#Precrocessing part, we are doing only stopword removal, stemming, and spelling correction if possible.
#process each file and do basic tasks, such as
#counting all words, mapping filenames
numToTitle=[]
titleToNum={}
list_all_words=[]
mappings={}
i=0
for filename in os.listdir():
    file = open(filename)
    title = file.readline()
    title = title.strip()
    numToTitle.append(title)
    titleToNum[title]=i
    i = i+1
    
    #feed all text data into a list, and count frequencies
    data_raw = file.read()
    
    data_raw = re.sub('ACT\s[1-9]', '', data_raw)
    data_raw = re.sub('Scene\s[1-9]', '', data_raw)
    data_raw = re.sub('\r?\n+', ' ', data_raw)

    temp_list = re.compile("=+").split(data_raw)
    full_data=""
    for partial_data in temp_list[3:len(temp_list)-1]:
        partial_data = re.sub('[^\w\s]', ' ', partial_data)
        partial_data = re.sub('\s+', ' ', partial_data)
        partial_data = re.sub('\s.\s', ' ', partial_data)
        partial_data = partial_data.lower()
        #at this point we have all the data extracted, cleaned
        #lowercase, no punctuation, no single characters
        full_data = full_data+partial_data

        #tokenizing the full text of this document
        tokenized_full_text = re.compile("\s").split(full_data)
    list_all_words += tokenized_full_text
    mappings[title] = tokenized_full_text

##30 most common words are removed as stopwords
freq = Counter(list_all_words)
stopwords=[]
for word, count in freq.most_common(30):
    stopwords.append(word)
#Now, we have the stopwords and our data with title in a dictionary
print("Stopwords removed:")
print(stopwords)

Stopwords removed:
['the', 'and', 'to', 'of', 'you', 'my', 'that', 'in', 'is', 'not', 'he', 'with', 'me', 'it', 'for', 'be', 'his', 'your', 'as', 'this', 'but', 'have', 'thou', 'him', 'will', 'so', 'what', 'her', 'all', 'do']


This is the most time taking task with complexity: O( D * (N+N) ).  
Hence, the complexity of the whole code would be: O( number of documents * average number of words in documents )  
Creating a new list with stopwords removed, and then stemming all of them  
Then assignning this cleaned and stemmed list of tokens back to the dictionary with title as key.  

In [11]:
# this block first chooses a document, then removes the stopword from it's text data
# then it creates a set of words containing stemmed versions, and removing duplicacies as well
# finally it puts back the cleaned and stemmed data back to the document's key in the dictionary

discarded=0
ps=PorterStemmer()
for title in mappings:

    # first removing stopwords
    clean_list = [word for word in mappings[title] if word not in stopwords]

    # now stemming the words
    stemmed_list = set()
    for word in clean_list:
        if word != '':
            stemmed_list.add(ps.stem(word))
    # converting the set to list
    final_clean_stemmed_list = list(stemmed_list)

    # assigning back the cleaned and stemmed list to dictionary
    mappings[title] = final_clean_stemmed_list
    

Creating postings with keys as word tokens, and value as a list of docIDs.  
Creating permuterm indices with adding a dollar sign and rotating the word.  
Try catch block if the key doesn't exist, to add it.  

In [12]:
# this block creates postings and permuterm indices from earlier cleaned and stemmed data
postings={}
permuterms={}
for i in range(0, len(numToTitle)):
    for word in mappings[numToTitle[i]]:
        dollar_word = word+'$'
        for n in range(len(dollar_word)):
            permuterms[dollar_word[n:] + dollar_word[:n]]=word
        try:
            if postings[word][len(postings[word])-1] != i:
                postings[word].append(i)
        except:
            postings[word]=[i]

Defining lambda expressions and functions needed for further processing.  
get_matched_keys (wildcard key with * at end): lambda expression to fetch keys which match with our wildcard searching format.  
get_wild (key entered by user in wildcard format): function to handle different cases of wildcard searching formats.  
sanitize_key (key entered by user): function to handle whether entered key has wildcard format or plain.  
listAND (list 1, list 2): function to join 2 lists with AND operator.  
listOR (list 1, list 2): function to join 2 lists with OR operator.  
listNOT (list 1): function to negate a list with NOT operator.  

In [13]:
# defining functions needed

#lambda expression to get wildcard word matchings from permuterm indices
get_matched_keys = lambda x: [pairs[1] for pairs in [(key, value) for key, value in permuterms.items() if key.startswith(x)]]

# function to handle the different cases of wildcard formats
def get_wild(key):
    
    if key[len(key)-1] == '*':
        if key[0] == '*':
            # *X* form look for X*
            x=key[1:-1]
        else:
            # X* form look for $X*
            x=key[:-1]
            x='$'+x
    elif key[0] == '*':
        # *X form look up X$*
        x=key[1:]
        x=x+'$'
    else:
        # X*Y form look up Y$X*
        x,y = key.split('*')
        x=y+'$'+x

    return get_matched_keys(x)

# function to check whether key needs wildcard processing or not (sent to stemming)
def sanitize_key(key):
    if '*' in key:
        return get_wild(key)
    else:
        return [ps.stem(key)]

#Functions for boolean AND, OR, NOT operations on lists
def listAND(list1, list2):
    i=0
    j=0
    answer=[]
    while i < (len(list1)) and j < (len(list2)):
        if(list1[i]==list2[j]):
            answer.append(list1[i])
            i=i+1
            j=j+1
        elif list1[i] < list2[j]:
            i=i+1
        else:
            j=j+1
    return answer

def listOR(list1, list2):
    i=0
    j=0
    answer=set()
    while i < (len(list1)):
        answer.add(list1[i])
        i=i+1
    while j < (len(list2)):
        answer.add(list2[j])
        j=j+1
    return list(answer)

def listNOT(list1):
    answer=[]
    for i in range(len(numToTitle)):
        if(i not in list1):
            answer.append(i)
    return answer

Block to just take input from the user.  
Input must be in bracket free form.  
Query first split on 'or' keyword, both sides are resolved and then added with listOR function.  

In [74]:
#process query, i.e. boolean expression parsing and wildcard searching in permuterm index
#queries do not support brackets as of now, can parse but need time

query = "br* and ca*ar or calpurnia"
print("Enter your query in bracket free form:")
#query = input()
query = query.lower()
query='not in'
query = query.split('or')



Enter your query in bracket free form:


Processing the parts recieved after OR splitting. Only 'and', 'not' keywords remain.   
Handling 'and' cases: either both keywords need to be negated, or only one of them needs to be negated, or none of them needs to be negated.  
If the part doesn't contain 'and' then we directly procees to check for NOT and process the keyword.  
Finally, join final list of lists with OR operator between all of them.

Remove duplicates and sort them and display.

In [75]:
orlists=[]
for part in query:
    part=part.strip()
    try:
        if 'and' in part:
            results=[]
            part = part.split(' ')
            if len(part)==3:
                #no not expression
                key_set_1 = sanitize_key(part[0])
                key_set_2 = sanitize_key(part[2])
                for key1 in key_set_1:
                    for key2 in key_set_2:
                        result = listAND(postings[key1], postings[key2])
                        if result != '':
                            orlists.append(result)
            elif len(part)==5:
                #both not expression
                key_set_1 = sanitize_key(part[1])
                key_set_2 = sanitize_key(part[4])
                for key1 in key_set_1:
                    for key2 in key_set_2:
                        result = listAND(listNOT(postings[key1]), listNOT(postings[key2]))
                        if result != '':
                            orlists.append(result)
            else:
                # one key has not operator
                if part[0]=='not':
                    key_set_1 = sanitize_key(part[1])
                    key_set_2 = sanitize_key(part[3])
                    for key1 in key_set_1:
                        for key2 in key_set_2:
                            result = listAND(listNOT(postings[key1]), postings[key2])
                            if result != '':
                                orlists.append(result)

                else:
                    key_set_1 = sanitize_key(part[0])
                    key_set_2 = sanitize_key(part[3])
                    for key1 in key_set_1:
                        for key2 in key_set_2:
                            result = listAND(postings[key1], listNOT(postings[key2]))
                            if result != '':
                                orlists.append(result)
        else:
            result=[]
            part = part.split(' ')
            if 'not' in part:
                key_set_1 = sanitize_key(part[1])
                print(key_set_1)
                for key in key_set_1:
                    result.extend(listNOT(postings[key]))
                    print(listNOT(postings[key]))
            else:
                key_set_1 = sanitize_key(part[1])
                #print(key_set_1)
                for key in key_set_1:
                    result.extend(postings[key])
                    #print(result)
            if result != '':
                orlists.append(result)
    except:
        print("Entered keys not found in the data, please search with another key.")

#now we have lists separated by OR operator
ans=[]
for list1 in orlists:
    for list2 in orlists:
        ans.extend(listOR(list1, list2))

ans = list(OrderedDict.fromkeys(ans))
ans.sort()
print(ans)

['in']
Entered keys not found in the data, please search with another key.
[]
